In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
import sys
import re

In [10]:
df = pd.read_csv("data_all.csv")
df.rename(columns={"match_id":"ids"},inplace=True)

In [11]:
df_type = pd.read_csv("type_of_match_all.csv")
df_type.set_index("ids",inplace=True)

In [12]:
df_all = df.join(df_type, on="ids")

In [13]:
df_all

,map,date,first_team_score,second_team_score,first_team_link,second_team_link,ids,isBoOne,match_num,ft_Maps played,...,st_pl_Rifle kills,st_pl_Sniper kills,st_pl_SMG kills,st_pl_Pistol kills,st_pl_Grenade,st_pl_Other,st_pl_players_in_team,lan,online,big_event
0,Cache,2019-04-18,12,16,/stats/teams/5973/Liquid,/stats/teams/5005/Complexity,84568,0,1,34.0,...,319.8,117.6,22.8,91.0,8.4,1.2,5,1,0,0
1,Dust2,2019-04-18,16,2,/stats/teams/5973/Liquid,/stats/teams/5005/Complexity,84569,0,2,34.0,...,319.8,117.6,22.8,91.0,8.4,1.2,5,1,0,0
2,Inferno,2019-04-18,16,3,/stats/teams/5973/Liquid,/stats/teams/5005/Complexity,84570,0,3,34.0,...,319.8,117.6,22.8,91.0,8.4,1.2,5,1,0,0
3,Mirage,2019-04-19,16,3,/stats/teams/5973/Liquid,/stats/teams/7801/Ghost,84678,0,1,37.0,...,308.2,133.4,39.0,93.8,5.6,9.4,5,1,0,0
4,Dust2,2019-04-19,16,10,/stats/teams/5973/Liquid,/stats/teams/7801/Ghost,84679,0,2,37.0,...,308.2,133.4,39.0,93.8,5.6,9.4,5,1,0,0
5,Nuke,2019-04-30,16,9,/stats/teams/5973/Liquid,/stats/teams/7567/BOOT-d%5BS%5D,85194,1,1,31.0,...,490.2,126.6,45.8,123.6,13.2,7.8,5,1,0,0
6,Dust2,2019-04-30,16,7,/stats/teams/5973/Liquid,/stats/teams/7532/BIG,85204,0,1,31.0,...,655.0,131.2,62.6,165.8,15.4,4.6,5,1,0,1
7,Overpass,2019-04-30,16,3,/stats/teams/5973/Liquid,/stats/teams/7532/BIG,85206,0,2,31.0,...,655.0,131.2,62.6,165.8,15.4,4.6,5,1,0,1
8,Nuke,2019-05-02,16,11,/stats/teams/5973/Liquid,/stats/teams/4411/NiP,85280,0,1,34.0,...,804.0,57.8,67.2,189.8,14.4,2.8,5,1,0,1
9,Overpass,2019-05-02,16,14,/stats/teams/5973/Liquid,/stats/teams/4411/NiP,85283,0,2,34.0,...,804.0,57.8,67.2,189.8,14.4,2.8,5,1,0,1


In [15]:
df_all = df_all[df_all['isBoOne']==0]


def get_team_id(team_link):
    return re.split("/", team_link)[3]


last_first_team = get_team_id(df_all.iloc[0]['first_team_link'])
last_second_team = get_team_id(df_all.iloc[0]['second_team_link'])
df_all['best_of_3_id'] = 0

df_all['ft_win_last_map'] = 0
df_all['st_win_last_map'] = 0

df_all['ft_prev_map_score'] = 0
df_all['st_prev_map_score'] = 0

counter = 0
df_all.head()
for idx,i in enumerate(df_all.index):
    first_team_id = get_team_id(df_all.iloc[idx]['first_team_link'])
    second_team_id = get_team_id(df_all.iloc[idx]['second_team_link'])
    if (df_all.iloc[idx]['match_num']-df_all.iloc[idx - 1]['match_num']) == 1 and (
            first_team_id == last_first_team) and (second_team_id == last_second_team):

        df_all.at[i, 'best_of_3_id'] = counter
        
        if df_all.iloc[idx - 1]['first_team_score']>df_all.iloc[idx - 1]['second_team_score']:
            df_all.at[i, 'ft_win_last_map'] = 1
        else:
            df_all.at[i, 'st_win_last_map'] = 1
        
        df_all.at[i, 'ft_prev_map_score'] = df_all.iloc[idx - 1]['first_team_score']
        df_all.at[i, 'st_prev_map_score'] = df_all.iloc[idx - 1]['second_team_score']

    else:
        counter += 1
        df_all.at[i, 'best_of_3_id'] = counter
        # print(df_all.iloc[i]['map'])
       
        last_first_team = get_team_id(df_all.iloc[idx]['first_team_link'])
        last_second_team = get_team_id(df_all.iloc[idx]['second_team_link'])


# df_all.head(15)
    
# df_all.drop("first_team_link",axis=1,inplace=True)
# df_all.drop("second_team_link",axis=1,inplace=True

/usr/lib/python3/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/lib/python3/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/usr/lib/python3/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.o

In [16]:
# Удаляю все битые матчи Б03 в которых не хватает матчей до полноценного Б03 (удалаяю недостатки парсера)
df_all = df_all.groupby("best_of_3_id").filter(lambda x: len(x) > 1)
df_all = df_all.groupby("best_of_3_id").filter(lambda x: len(x) < 4)
df_all = df_all.groupby("best_of_3_id").filter(lambda x: x['match_num'].sum() == 6 or x['match_num'].sum() == 3)



In [18]:
# Drop date and teams link and
df_all.drop("date",axis=1,inplace=True)
df_all.drop("first_team_link",axis=1,inplace=True)
df_all.drop("second_team_link",axis=1,inplace=True)


In [19]:
le = preprocessing.LabelEncoder()

In [20]:
df_all['map'] = le.fit_transform(df_all['map'])

In [38]:
df_all['ft_age'].fillna(df_all.ft_age.mean(),inplace=True)
df_all['st_age'].fillna(df_all.st_age.mean(),inplace=True)

In [52]:
df_all.to_csv("clear_bo3_file.csv")

In [53]:
df_all_bo1 = df.join(df_type, on="ids")

In [56]:
df_all_bo1 = df_all_bo1[df_all_bo1['isBoOne']==1]

57

In [57]:
df_all_bo1['ft_age'].fillna(df_all.ft_age.mean(),inplace=True)
df_all_bo1['st_age'].fillna(df_all.st_age.mean(),inplace=True)

map                                        243
date                                       243
first_team_score                           243
second_team_score                          243
first_team_link                            243
second_team_link                           243
ids                                        243
isBoOne                                    243
match_num                                  243
ft_Maps played                             243
ft_Win Rate                                243
ft_Total kills                             243
ft_Total deaths                            243
ft_Rounds played                           243
ft_K/D Ratio                               243
ft_age                                     243
ft_mp_Times played                         243
ft_mp_Total rounds played                  243
ft_mp_Rounds won                           243
ft_mp_Win percent                          243
ft_mp_Pistol rounds                        243
ft_mp_Pistol 